In [1]:
import torch.nn as nn
import torch.optim as optim
from classes import SpeedEstimatorRNN, VehicleSpeedDataset
from torch.utils.data import DataLoader

In [2]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! You can use a GPU for training.")
    print("Number of GPUs available:", torch.cuda.device_count())
    print("Current GPU being used:", torch.cuda.current_device())
    print("GPU Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("CUDA is not available. Training will be performed on the CPU.")

CUDA is available! You can use a GPU for training.
Number of GPUs available: 1
Current GPU being used: 0
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
# Set dataset path
data_path = r"C:\my files\thesis\AI_training\training_data\training_data\training_data_2"
extension = "*.csv"

# Hyperparameters that will alter throughout the model creations
input_size = 14  # Number of CAN signals per timestep
hidden_size = [64, 64, 64, 128, 128, 256, 128, 128, 128, 128, 128, 128]
num_layers = [1, 2, 3, 1, 3, 3, 2, 2, 2, 2, 4, 3]
learning_rate = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.005, 0.001, 0.001, 0.0001, 0.0005, 0.001]
# num of sequences in one batch
batch_size = [64, 64, 64, 64, 64, 64, 64, 128, 64, 64, 64, 32]
dropout_rate = [0, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.4, 0.2]
sequence_length = [200, 200, 200, 200, 200, 200, 200, 200, 400, 50, 200, 200]


# parameters of the simulation
step_size = 10 # what the overlap between the sequences should look like in the extracted dataset
output_size = 1
num_epochs = 15

num_models = 12

default_location = r"C:\my files\thesis\AI_training\trained_models\iteration_2\model_"

In [ ]:
# Training loops
for j in range(num_models):
    # Load dataset and DataLoader
    dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[12], step_size = step_size)
    dataloader = DataLoader(dataset, batch_size=batch_size[j], shuffle=True, num_workers= 6, pin_memory=True)

    # Initialize model, loss function, and optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SpeedEstimatorRNN(input_size, hidden_size[j], num_layers[j], output_size).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[j])

    for epoch in range(num_epochs):
        total_loss = 0

        for batch_idx, (features, speeds) in enumerate(dataloader):
            features, speeds = features.to(device), speeds.to(device)

            # Forward pass
            outputs = model(features)
            loss = criterion(outputs, speeds)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "sequence_length": sequence_length,
        "input_size": input_size,
        "hidden_size": hidden_size,
        "num_layers": num_layers,
        "output_size": output_size,
        "learning_rate": learning_rate,
        "num_epochs": num_epochs
    }, default_location + str(j))

#12th model should still be executed

In [4]:
# Load dataset and DataLoader
dataset = VehicleSpeedDataset(data_path, extension, seq_length = sequence_length[11], step_size = step_size)
dataloader = DataLoader(dataset, batch_size=batch_size[11], shuffle=True, num_workers= 6, pin_memory=True)

# Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpeedEstimatorRNN(input_size, hidden_size[11], num_layers[11], output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate[11])

for epoch in range(num_epochs):
    total_loss = 0

    for batch_idx, (features, speeds) in enumerate(dataloader):
        features, speeds = features.to(device), speeds.to(device)

        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, speeds)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}")

torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "sequence_length": sequence_length,
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_layers": num_layers,
    "output_size": output_size,
    "learning_rate": learning_rate,
    "num_epochs": num_epochs
}, default_location + str(12))

Epoch [1/15], Loss: 12.3217
Epoch [2/15], Loss: 0.7816
Epoch [3/15], Loss: 0.8619
Epoch [4/15], Loss: 0.5878
Epoch [5/15], Loss: 0.6295
Epoch [6/15], Loss: 0.5368
Epoch [7/15], Loss: 0.6382
Epoch [8/15], Loss: 0.4038
Epoch [9/15], Loss: 0.4108
Epoch [10/15], Loss: 0.4844
Epoch [11/15], Loss: 0.4186
Epoch [12/15], Loss: 0.3590
Epoch [13/15], Loss: 0.3533
Epoch [14/15], Loss: 0.4199
Epoch [15/15], Loss: 0.3207
